In [ ]:
import csv  # Para leer y escribir archivos CSV
import os  # Para operaciones en el sistema de archivos como verificar rutas
import requests  # Para hacer solicitudes HTTP
from bs4 import BeautifulSoup  # Para el análisis HTML
import time  # Para pausas entre solicitudes



# Archivo para almacenar las URL visitadas
visited_urls_file = "visited_urls.csv"

# Archivo CSV que contiene las URL de inicio para rastrear
start_urls_csv = "mafia_nigeriana_2009-02-24_2024-01-01.csv"

# Clase Spider para manejar el rastreo de artículos de noticias
class MySpider:
    def __init__(self):
        pass

    # Método para iniciar las solicitudes
    def start_requests(self):

        # Verifica si existe el archivo de URL visitadas para ver si es la primera ejecución o una continuación después de un fallo
        if os.path.exists(visited_urls_file):
            # Abre el archivo y crea un conjunto de URL ya visitadas
            with open(visited_urls_file) as f:
                reader = csv.reader(f)
                visited_urls = {row[0] for row in reader if row}
        else:
            # Si el archivo no existe, es la primera ejecución
            visited_urls = set()

        # Abre el CSV de URL de inicio y itera a través de él
        with open(start_urls_csv, "r", newline="") as file:
            reader = csv.DictReader(file)

            iteration = 1

            for row in reader:
                url = row["URL"]

                print(f"Iteración número {iteration}")

                # Verifica la URL contra las ya visitadas
                if url not in visited_urls:
                    visited_urls.add(url)

                    # Hace la solicitud HTTP
                    response = requests.get(url)
                    if response.status_code == 200:
                        # Llama al método parse para manejar la respuesta
                        self.parse(response)
                    elif response.status_code == 403:

                        print('status is 403\n' * 3)

                        print('start the 4 minute wait')

                        for i in range(1,5):
                            time.sleep(60)
                            print(f'{i} minutes have passed')
                iteration += 1


    # Método que maneja el análisis de la respuesta
    def parse(self, response):
        # Analiza el contenido HTML
        soup = BeautifulSoup(response.content, "html.parser")

        # Encuentra todos los elementos de artículo en la página
        articles = soup.find_all("article")

        # Lista para almacenar los datos extraídos del artículo
        article_data = []

        for article in articles:
            # Obtiene el HTML interno de la etiqueta de anclaje del título
            anchor_html = article.find("h1").find("a")

            # Extrae el texto limpio del título
            title = anchor_html.get_text(separator=" ", strip=True)

            # Extrae el enlace del artículo
            link = anchor_html["href"]

            # +++Extrae la fecha de publicación+++

            # Encuentra el elemento <aside>
            aside_element = article.find("aside").find_all("a")

            date = aside_element[-1].get_text(separator=" ", strip=True)



            # +++fin de extraer la fecha de publicación+++

            # Agrega los datos del artículo a la lista
            article_data.append(
                {"title": title, "link": link, "date": date, "page_url": response.url}
            )

        # Escribe los datos extraídos del artículo en el archivo CSV
        with open("output_test.csv", "a", newline="", encoding="utf-8") as file:
            writer = csv.DictWriter(
                file, fieldnames=["title", "link", "date", "page_url"]
            )

            # Escribe el encabezado si el archivo está vacío
            if os.stat("output_test.csv").st_size == 0:
                writer.writeheader()

            # Escribe una fila para cada dato del artículo
            for data in article_data:
                writer.writerow(data)

        # Agrega la URL al archivo de visitados
        with open(visited_urls_file, "a", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow([response.url])

    # Método llamado cuando se cierra el Spider
    def closed(self, reason):
        # Registra la razón del cierre del spider
        print("Spider cerrado:", reason)

    # Método para manejar excepciones de proceso
    def process_exception(self, request, exception, spider):
        if isinstance(exception, requests.exceptions.HTTPError) and exception.response.status_code == 403:
            # Registra el error 403
            print(f"Recibido error 403 en {request.url}")

            # Espera 6 minutos antes de volver a intentar la solicitud
            print("Esperando 6 minutos antes de volver a intentar la solicitud...")
            time.sleep(360)
            return requests.get(request.url)


In [ ]:

# Instancia del Spider
spider = MySpider()
# Inicia las solicitudes
spider.start_requests()
